In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
osu_bench_np2_get = pd.read_csv("2023-07-21_weaver3_osu_bench_np2_get.csv")
osu_bench_np2_put = pd.read_csv("2023-07-21_weaver3_osu_bench_np2_put.csv")
dfs = [osu_bench_np2_get, osu_bench_np2_put]
labels = ["osu_bench_np2_get", "osu_bench_np2_put"]

In [ ]:
def figure_for_attribute(dfs,
                         labels,
                         attribute,
                         xaxis="n",
                         xlabel="problem size (array of size n)",
                         ylabel = None,
                         title = None,
                         style="--",
                         scale="linear",
                         figsize=[8.25,4.95],
                         colors=None):
    color_defaults = ["tab:purple", "tab:brown", "tab:red", "k", "g", "c", "tab:orange", "tab:blue", "y"]
    if colors is None:
        colors = color_defaults[:len(dfs)]
    
    
    if ylabel is None:
        ylabel = attribute
    
    if title is None:
        title = f"update benchmark ({attribute.replace('_',' ')})"
    
    if len(dfs) == 0:
        print("required to have at least one line to plot")
    
    maxy = max(dfs[0][f"{attribute}-mean"])
    miny = min(dfs[0][f"{attribute}-mean"])
    for df in dfs:
        cur_max = max(df[f"{attribute}-mean"])
        cur_min = min(df[f"{attribute}-mean"])
        if cur_max > maxy:
            maxy = cur_max
        if cur_min < miny:
            miny = cur_min
    
    if labels is None:
        print("required to label each line")
        
    plt.figure(figsize=figsize)
    for df, dflabel,c in zip(dfs,labels,colors):
        plt.plot(df[xaxis], df[f"{attribute}-mean"], style, label=dflabel, color=c)
    plt.title(title)
    plt.legend()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if type(scale) is str:
        plt.xscale(scale)
        plt.yscale(scale)
    else:
        plt.xscale(scale[0])
        plt.yscale(scale[1])
    if scale == "linear":
        ylim_base = 0
    else:
        ylim_base = miny*0.95
    plt.ylim(ylim_base,(maxy)*1.05)
    plt.grid()
    plt.savefig(f"{title}.png")
    plt.show()

In [ ]:
figure_for_attribute(dfs, labels, "message_rate", "message rate (GB/s)", "2-node rma osu benchmark message rate", style="*-", scale="log")

In [ ]:
figure_for_attribute(dfs, labels, "time_latency", "message latency (microseconds)", "2-node rma osu benchmark message latency", style="*-", scale="log")

# 2023-07-26
- Paper submission
    - no extended abstract
    - http://nowlab.cse.ohio-state.edu/espm2/ an option
    - ISC?
    - IPDPS?
    - ExaMPI (SC)?
- Year-round Internship logistics?
    - If a student is supported by a project proposal, all good
    - Otherwise, agreement is to work together with subcontract
    - Otherwise, I would have to independently meet PhD requirements and Sandia requirements
- OSU a true test?
    - MPI fence is a synchronization (MPI barrier would be enough)
    - fence is just memory ordering (Like the mailbox)
        - So put fence isn't enough to synchronize processes
    - Must barrier between processes
    - (TODO: read nvshmem documentation especially nvshmem fence)
        - This means the correct approach would be quiet barrier
    - (PGAS is quite slow when it comes to measure this latency)
    - If I was to implement a check after the send, it might not succeed
    - To test: initialize the buffer to some value on the sender with a seed, 0 on receiver
        - then do ping-pong, and check that the data is exactly the data we send
        - So we will receive the zeroed data, maybe a little bit of the data we put in
    - So with the latency, we might not actually be enqueueing them
    - Documentation: https://docs.nvidia.com/nvshmem/archives/nvshmem-101/api/docs/gen/api/ordering.html
    - += vs =
        - If I don't have the opposite operation, I could just go to the next stage
- A long discussion about Allreduce